In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ../TileEM/

/home/jlee782/crowd-seg/analysis/TileEM


In [3]:
from TileEM_plot_toolbox import *
from qualityBaseline import *

In [12]:
from glob import glob

In [17]:
worker_Nbatches={5:10,10:8,15:6,20:4,25:2,30:1}

In [19]:
sampleN_lst=worker_Nbatches.keys()

In [22]:
for Nworker in sampleN_lst:
    for batch_num in range(worker_Nbatches[Nworker]):
        dir_name = "{0}worker_rand{1}".format(Nworker,batch_num)
        print dir_name
        

5worker_rand0
5worker_rand1
5worker_rand2
5worker_rand3
5worker_rand4
5worker_rand5
5worker_rand6
5worker_rand7
5worker_rand8
5worker_rand9
10worker_rand0
10worker_rand1
10worker_rand2
10worker_rand3
10worker_rand4
10worker_rand5
10worker_rand6
10worker_rand7
15worker_rand0
15worker_rand1
15worker_rand2
15worker_rand3
15worker_rand4
15worker_rand5
20worker_rand0
20worker_rand1
20worker_rand2
20worker_rand3
25worker_rand0
25worker_rand1
30worker_rand0


In [74]:
list(set(df.object_id))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47]

In [87]:
# Summarization-Based Method 
# Creating 2 empty precision recall table containing P,R for each metric used 
cols = [u'Num Points',u'Area Ratio',u'Jaccard [Self]', u'Precision [Self]', u'Recall [Self]']
index = list(set(df.object_id))

precision_df = pd.DataFrame(index=index,columns=cols)
recall_df = pd.DataFrame(index=index,columns=cols)

In [56]:
cd ../../

/home/jlee782/crowd-seg/analysis/TileEM


In [57]:
from collections import OrderedDict
df = pd.read_csv("../computed_my_COCO_BBvals.csv",index_col=0)

In [58]:
os.chdir("sample/5worker_rand0")

In [83]:
for objid in object_lst:
    workers=pkl.load(open("worker{}.pkl".format(objid)))
    filtered_df = df[df["worker_id"].isin(workers)] #only look at summarization scores of sampled workers
    filtered_df.ix[filtered_df["Precision [Self]"]>1,"Precision [Self]"]=1
    filtered_df.ix[df["Recall [Self]"]>1,"Recall [Self]"]=1
    df_noBBG = filtered_df[~(filtered_df["worker_id"].isin([1,2,3]))] #Filter our ground truth workers
    #attr="Jaccard [Self]"# select best BB based on the worker with the highest scoring  atttr metric
    for attr in cols: 
        idx = df_noBBG.groupby("object_id")[attr].transform(max) == df_noBBG[attr]
        best_BBs = df_noBBG[idx] #best BBs based on these metrics
    #     if len(best_BBs)!= len(workers):
        #Average our the percision recall of duplicate best BBs 
        best_BBs = best_BBs.groupby("object_id")["Recall [Self]","Precision [Self]"].mean()#.reset_index()
        best_BBs_recall = list(best_BBs["Recall [Self]"])
        best_BBs_precision =  list(best_BBs["Precision [Self]"])
#         #try:
#         #add it to the large table 
        precision_df[attr] = best_BBs_precision
        recall_df[attr] = best_BBs_recall
#         except(ValueError):
#             print attr
#             print len(best_BBs_recall)
#             print len(best_BBs_precision)

ValueError: Length of values does not match length of index

In [90]:
len(best_BBs_precision)

46

In [91]:
len(precision_df[attr])

47

In [84]:
precision_df[attr]

0     0.187262
1     0.820162
2     0.930130
3     0.922176
4     0.937226
5     0.873172
6     0.867883
7     0.601454
8     0.946964
9     0.969249
10    0.840968
11    0.888776
12    0.951116
13    0.941721
14    0.591902
15    0.916546
16    0.852871
17    0.965686
18    0.934827
19    0.892106
20    0.948942
21    0.828418
22    0.925933
23    0.913324
24    0.792764
25    0.965637
26    0.961400
27    0.864391
28    0.855191
29    0.825963
30    0.773471
31    0.966971
32    0.793831
33    0.984807
34    0.159437
35    0.975662
36    0.910498
37    0.972935
38    0.984286
39    0.018709
40    0.023669
41    0.929278
42    0.960470
43    0.907802
44    0.899235
45    0.940679
46    0.891945
Name: Num Points, dtype: float64

In [79]:
len(best_BBs)

46

In [81]:
len(best_BBs_precision)

46

In [52]:
len(best_BBs_recall)

0

In [43]:
len(workers)

5

In [42]:
len(best_BBs)

45

In [96]:
visionPR = pd.read_csv("../../Image-Segmentation/Vision-stuff/vision-baseline/PR.csv")

In [97]:
precision_df["Vision"] = visionPR["precision"]
recall_df["Vision"] = visionPR["recall"]

In [101]:
precision_df.mean()

Num Points          0.707535
Area Ratio          0.559025
Jaccard [Self]      0.954470
Precision [Self]    0.979881
Recall [Self]       0.729465
Vision              0.905204
dtype: float64

In [99]:
recall_df.mean()

Num Points          0.843395
Area Ratio          0.870589
Jaccard [Self]      0.950287
Precision [Self]    0.757573
Recall [Self]       0.989135
Vision              0.736919
dtype: float64

In [103]:
from tabulate import tabulate

In [134]:
print [list(precision_df.mean().as_matrix()),list(recall_df.mean().as_matrix())]

[[0.70753547589361709, 0.5590247110212766, 0.95446982951063797, 0.97988143444680853, 0.72946534988747957, 0.90520449921850243], [0.84339502145744694, 0.87058918721276601, 0.95028722610638316, 0.75757326877912845, 0.98913499846808495, 0.73691887132975531]]


In [143]:
tbl = pd.DataFrame([list(precision_df.mean().as_matrix()),list(recall_df.mean().as_matrix())],columns=precision_df.keys(),index=["Precision","Recall"])

In [146]:
tbl

,Num Points,Area Ratio,Jaccard [Self],Precision [Self],Recall [Self],Vision
Precision,0.707535,0.559025,0.954470,0.979881,0.729465,0.905204
Recall,0.843395,0.870589,0.950287,0.757573,0.989135,0.736919


In [144]:
# keys = list(precision_df.keys())
# keys.insert(0,"Metric")
# p_lst = list(precision_df.mean().as_matrix())
# p_lst.insert(0,"Precision")
# r_lst = list(recall_df.mean().as_matrix())
# r_lst.insert(0,"Recall")
# tbl=[keys,p_lst,r_lst]
# print tabulate(tbl,tablefmt='latex',floatfmt='.2f')

In [123]:
# tabulate(grouped_ji_fit[idx],showindex="False",headers='keys',tablefmt='latex',floatfmt='.2f')

In [147]:
print tabulate(tbl,tablefmt='latex',headers='keys',floatfmt='.2f')

\begin{tabular}{lrrrrrr}
\hline
           &   Num Points &   Area Ratio &   Jaccard [Self] &   Precision [Self] &   Recall [Self] &   Vision \\
\hline
 Precision &         0.71 &         0.56 &             0.95 &               0.98 &            0.73 &     0.91 \\
 Recall    &         0.84 &         0.87 &             0.95 &               0.76 &            0.99 &     0.74 \\
\hline
\end{tabular}


# Need to modify so that these are computed based on the selected sampled workers!